In [1]:
import pandas as pd
import numpy as np 

In [2]:
temp_train = pd.read_csv("C:/program files/Anaconda/CsvFiles/twitter_train.csv")
temp_test = pd.read_csv("C:/program files/Anaconda/CsvFiles/twitter_test.csv")

In [16]:
Y_train = temp_train['airline_sentiment']

In [113]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

words_to_remove = stopwords.words("english") + list(punctuation)

def part_of_speech(postag):
    if postag[0] =='N':
        return 'n'
    elif postag[0] == 'J':
        return 'a'
    elif postag[0] == 'V':
        return 'v'
    else:
        return 'n'

def clean_data(text):
    op_list = []
    text_list = word_tokenize(text)[2:]
    for w in text_list:
        if w not in words_to_remove:
            pos = pos_tag([w])[0][1]
            lemmatized_word = WordNetLemmatizer().lemmatize(w,part_of_speech(pos))
            op_list.append(lemmatized_word)
    final_text = " ".join(op_list)
    return final_text

In [114]:
training_text = temp_train['text'].values

In [115]:
training_text = list(training_text)
testing_text = list(temp_test['text'].values)
train_for_cv = [clean_data(t) for t in training_text]
test_for_cv = [clean_data(t) for t in testing_text]

In [142]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(max_features = 500 , ngram_range = (1,3))
X_train = count_vec.fit_transform(train_for_cv)
X_test = count_vec.transform(test_for_cv)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
grid = {"C":[1,10,100,1000,5000,10000] , "max_iter" : [100,200,300,400,500]}
grid2 = {"C":[1,10,100,1000,5000,10000] , "gamma":[1e-1 , 1e-2, 1e-3 , 5e-2]}
clf1 = SVC()
clf2 = RandomForestClassifier()
clf3 = LogisticRegression()
abc1 = GridSearchCV(clf1 , grid2)
abc2 = GridSearchCV(clf3 , grid)
abc1.fit(X_train , Y_train)
abc2.fit(X_train , Y_train)


C:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-r

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10, 100, 1000, 5000, 10000],
                         'max_iter': [100, 200, 300, 400, 500]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [146]:
print(abc1.best_estimator_) #svc
print(abc2.best_estimator_) # logistic

clf2.fit(X_train , Y_train)

Ypred_randomForest = clf2.predict(X_test)


SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
np.savetext('ansRandomFor.csv' , Ypred_randomForest)